<a href="https://colab.research.google.com/github/FlorianShepherd/ml/blob/main/nlp_beginner_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"flowshepherd","key":"c8afa3634896a4e845cc7f22a7e82218"}\n'}

In [3]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle competitions download -c nlp-getting-started

nlp-getting-started.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!mkdir Dataset
!cp /content/nlp-getting-started.zip /content/Dataset/nlp-getting-started.zip
!unzip -q /content/Dataset/nlp-getting-started.zip -d /content/Dataset
!rm /content/Dataset/nlp-getting-started.zip


mkdir: cannot create directory ‘Dataset’: File exists
replace /content/Dataset/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
! pip install --upgrade pip
! pip install --no-cache-dir transformers
! pip install sentencepiece
! pip install accelerate
! pip install datasets

In [ ]:
train_df = pd.read_csv("/content/Dataset/train.csv")
test_df = pd.read_csv("/content/Dataset/test.csv")

In [ ]:
target_group = train_df.groupby(["keyword", "target"]).count()
target_group["text"].iloc[:10].plot.bar(stacked=True)

In [ ]:
# data set is very imbalanced within categories, but not imbalanced in the overall distribution (roughly 50:50 in 0:1 targets)
target_group

In [ ]:
# how many keywords are nan? -> 0.8% - just drop them?
train_df[train_df.keyword.isna()].count() / train_df[~train_df.keyword.isna()].count() * 100.

# drop them
train_df = train_df[~train_df.keyword.isna()]

In [ ]:
# how many keywords are nan? Sample text
# train_df[train_df.keyword.isna()].iloc[60]["text"]

In [ ]:
# very similar strings ("Obama bla") and also only one sample without being target == 1 -> Ideas: Remove http://t.co ends and then remove duplicates? Also even out samples per keyword
train_df[train_df["keyword"] == "aftershock"]

In [ ]:
result_df = list()

for keyword, subdf in train_df.groupby("keyword"):
  ones = subdf["target"].sum()
  zeros = len(subdf) - ones

  if ones == 0 or zeros == 0:
    # print(f"For the keyword {keyword} - we have {ones} ones and {zeros} zeros")
    result_df.append(subdf)
    continue

  df_zero = subdf[subdf.target == 0.]
  df_one = subdf[subdf.target == 1.]

  if ones > zeros:
    # duplicates zeros
    n_times = np.floor(len(df_one) / len(df_zero))
    n_rand = len(df_one) % len(df_zero)

    result_df.append(df_zero.loc[df_zero.index.repeat(n_times)])
    result_df.append(df_zero.loc[np.random.choice(df_zero.index, n_rand)])
    result_df.append(df_one)

  elif zeros > ones:
    # duplicates ones
    n_times = np.floor(len(df_zero) / len(df_one))
    n_rand = len(df_zero) % len(df_one)

    result_df.append(df_one.loc[df_one.index.repeat(n_times)])
    result_df.append(df_one.loc[np.random.choice(df_one.index, n_rand)])
    result_df.append(df_zero)

  else:
    # equal - append subdf
    result_df.append(subdf)


result_df = pd.concat(result_df)

train_df = result_df

# all bars should be equal now except the ones where we only have one category
target_group = result_df.groupby(["keyword", "target"]).count()
target_group["text"].iloc[:10].plot.bar(stacked=True)

In [ ]:
np.random.choice(train_df.iloc[:10].index, 12)
np.floor(26/10)
subdf.loc[subdf.index.repeat(3)]

In [ ]:
train_df["target"] = train_df["target"].astype(float)

mode = train_df.mode().iloc[0]
train_df.fillna(mode, inplace=True)
train_df.isna().sum()

In [ ]:
train_df.iloc[3]

In [ ]:
train_df["input"] = "TEXT1: " + train_df["keyword"] + "; TEXT3: " + train_df["text"]
# train_df["input"] = "TEXT2: " + train_df["location"] + "; TEXT3: " + train_df["text"]

In [ ]:
train_df["text"]

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
model_nm = 'microsoft/deberta-v3-small'
# model_nm = 'microsoft/DeBERTa-v3-base'
tokz = AutoTokenizer.from_pretrained(model_nm, use_fast=False)

In [ ]:
tokz.tokenize(train_df.iloc[0]["input"])

In [ ]:
def tok_func(x): return tokz(x["input"])

In [ ]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(train_df)

In [ ]:
ds

In [ ]:
tok_ds = ds.map(tok_func, batched=True)

In [ ]:
tok_ds

In [ ]:
row = tok_ds[0]
row['input'], row['input_ids']

In [ ]:
tok_ds = tok_ds.rename_columns({'target':'labels'})

In [ ]:
dds = tok_ds.train_test_split(0.25, seed=1337)

In [ ]:
mode = test_df.mode().iloc[0]
test_df.fillna(mode, inplace=True)
test_df.isna().sum()

test_df["input"] = "TEXT1: " + test_df["keyword"] + "; TEXT2: " + test_df["location"] + "; TEXT3: " + test_df["text"]
# test_df["input"] = "TEXT2: " + test_df["location"] + "; TEXT3: " + test_df["text"]


eval_ds = Dataset.from_pandas(test_df).map(tok_func, batched=True)

In [ ]:
eval_ds

In [ ]:
def mse(x,y): return np.mean((x-y)**2)
def mse_d(eval_pred): return {'mse': mse(*eval_pred)}

def acc(x,y): return np.mean(1.0-np.abs(np.round(x)-np.round(y)))
def acc_d(eval_pred): return {'acc': acc(*eval_pred)}

acc(0, 1)

In [ ]:
from transformers import TrainingArguments,Trainer


In [ ]:
bs = 128
epochs = 5
lr = 8e-5
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=acc_d)

In [ ]:
trainer.train();

In [ ]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds[preds>=0.5] = 1.
preds[preds<0.5] = 0.

In [ ]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'target': preds.astype(int).T[0].tolist()
})

submission.to_csv('submission.csv', index=False)


In [ ]:
preds.astype(int).T[0].tolist()[:10]

In [ ]:
dds["test"]["labels"][:10]

In [ ]:
preds = trainer.predict(dds["test"]).predictions.astype(float)
preds[preds>=0.5] = 1.
preds[preds<0.5] = 0.
preds[10:20]